# Exp011: Corpus Rule Detection
For a good external validation, each rule is searched for in the corpus and marked if there is at least 80% precision among the hits. If there are less than 5 hits in the first ten examples, the rule is skipped, otherwise 10 more appear.

In [25]:
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
import os
import random
import sys
sys.path.append('../source')
import models
import data
import importlib
importlib.reload(models)

<module 'models' from '/mnt/qb/work/meurers/mpb672/grammarctg/experiments/../source/models.py'>

Load datasets

In [31]:
# egp instances
egp_examples = pd.read_json("../data/egp_examples.json")
# load corpus sentences and prepare dataloader
sentences = data.get_mixed_sentences(20000)
encoded_inputs = models.bert_tokenizer(sentences, return_tensors='pt', max_length=64, padding='max_length', truncation=True)
dataset = TensorDataset(encoded_inputs['input_ids'], encoded_inputs['attention_mask'])
dataloader = DataLoader(dataset, batch_size=64, shuffle=False)
# output dataset
output_path = '../data/coded_corpus_hits.json'
coded_instances = pd.DataFrame(columns=['#', 'sentence', 'correct', 'score', 'max_token']) if not os.path.exists(output_path) else pd.read_json(output_path)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:26<00:00,  8.86s/it]


for each rule:
- train classifier with existing dataset
- search corpus
- output rule to user
- ask for user input until the precision is clear enough

In [19]:
def get_trained_classifer(rule, egp_examples):
    classifier = models.RuleDetector(models.bert_encoder)
    positive = rule['augmented_examples']
    negative = rule['augmented_negative_examples']
    others = [example for sublist in egp_examples.loc[egp_examples['#'] != rule['#'], 'augmented_examples'].to_list() for example in sublist]
    
    dataset = data.get_dataset(positive, negative, others, models.bert_tokenizer, 64) 
    train_dataloader, val_dataloader = data.get_loaders(dataset)
    
    optimizer, accuracy = models.train(classifier, train_dataloader, val_dataloader, num_epochs=3)
    return classifier

In [48]:
def criterion(coded_instances, min_precision=0.8, num_rules=1):
    correct_per_rule = coded_instances.groupby('#')['correct'].mean()
    return len(correct_per_rule[correct_per_rule > min_precision]) > num_rules

In [49]:
while not criterion(coded_instances, num_rules=19):
    rule = egp_examples.sample(1).iloc[0] # sample random rule
    if rule['#'] in coded_instances['#']: continue
    print(f"{rule['type']}: {rule['Can-do statement']} ({rule['SuperCategory']}: {rule['SubCategory']})")
    print(rule['Example'])
    
    classifier = get_trained_classifer(rule, egp_examples)
    scores, tokens = models.score_corpus(classifier, dataloader, max_positive=250, max_batches=1250)
    results = [(score, token, sample) for score, token, sample in zip(scores, tokens, sentences[:len(scores)]) if score > 0.5]
    sorted_results = iter(sorted(results, key=lambda x: x[0], reverse=True))
    while len(coded_instances[coded_instances['#'] == rule['#']]) < 20:
        if len(coded_instances[coded_instances['#'] == rule['#']]) >= 10: # check if we have at least 40% after 10 instances
            if not criterion(coded_instances[coded_instances['#'] == rule['#']], 0.4, 0): break
        score, token, sample = next(sorted_results)
        user_response = input(f"{sample}")
        new_row = pd.DataFrame({'#': [rule['#']],
             'sentence': [sample],
             'correct': [True if user_response == '2' else False],
             'score': [score],
             'max_token': [token]})
        coded_instances = pd.concat([coded_instances, new_row], ignore_index=True)
        coded_instances.to_json(output_path)

FORM/USE: Can use the negative question form as a persuasion strategy. (PRESENT: present simple)
Don't you just hate taking the bus to school every morning? well I have a perfect solution for you, a great bike with good brakes, a bell and lights. it is a great bike but since I never use it I would be happy to sell it to you. 

Don't you find that when you are having a shower or bath, you occasionally run out of water?


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:01<00:00, 17.61it/s]


Training loss: 0.3339063185453415
Accuracy: 0.985


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:01<00:00, 18.05it/s]


Training loss: 0.08209181800484658
Accuracy: 0.995


...

USE: Can use the present simple with a wide range of reporting verbs, especially in academic contexts, including 'demonstrate', 'illustrate'. (PRESENT: present simple)
The popularity of this TV game in Russia clearly demonstrates the nature of human fears and dreams. 

The 2 charts illustrate the number of employees, and the trends in profit for three factories, namely the factories located in London, Leeds, and Bristol, which belong to the same company, during the year 2003.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:01<00:00, 18.55it/s]


Training loss: 0.5689870703220368
Accuracy: 0.82


 12%|████████████████▌                                                                                                                         | 3/25 [00:00<00:01, 17.89it/s]


KeyboardInterrupt: 

In [52]:
correct_per_rule = coded_instances.groupby('#')['correct'].mean()
len(correct_per_rule[correct_per_rule > 0.8])

10